In [2]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import seaborn as sns 
import statsmodels.stats.proportion as proportion
from scipy.stats import ttest_ind,mannwhitneyu,shapiro,norm
from statsmodels.stats.weightstats import ztest
from tqdm import tqdm
import timeit
from scipy import stats
import math
from datetime import date, datetime, timedelta
import time
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import clickhouse_connect  

In [3]:
import gspread
from gspread_dataframe import get_as_dataframe
from google.oauth2.service_account import Credentials
import requests
import xml.etree.ElementTree as ET
from oauth2client.service_account import ServiceAccountCredentials

# Код для прогрузки диапазона дат

In [9]:
# Функция для получения курса йены  
def get_jpy_rate(date):  
    formatted_date = date.strftime("%d/%m/%Y")  
    url = f"https://www.cbr.ru/scripts/XML_daily.asp?date_req={formatted_date}"  
    response = requests.get(url)  
    response.raise_for_status()  
    
    tree = ET.fromstring(response.content)  
    for currency in tree.findall('Valute'):  
        char_code = currency.find('CharCode').text  
        if char_code == "JPY":  
            rate = currency.find('Value').text   
            return float(rate.replace(',', '.')) / 100  # Преобразуем строку в число с плавающей запятой  
    return None  # Если курс не найден, возвращаем None  

def write_to_google_sheets(sheet_name, date, value):
    # Авторизация Google Sheets API
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("exchange-rate.json", scope)
    client = gspread.authorize(creds)

    # Открытие нужного Google Sheets файла
    spreadsheet = client.open(sheet_name)
    
    # Открытие листа по имени 'exchange_rate'
    sheet = spreadsheet.worksheet("exchange_rate")
    
    # Преобразование даты в формат 'YYYY-MM-DD'
    formatted_date = date.strftime("%Y-%m-%d")
    sheet.append_row([formatted_date, value], value_input_option="USER_ENTERED")


# Основной блок  
if __name__ == "__main__":  
    try:  
        start_date = datetime(2024, 10, 1)  # Начальная дата  
        end_date = datetime.now()  # Текущая дата  

        # Проходим по датам от 01/01/2025 до текущей даты  
        current_date = start_date  
        while current_date <= end_date:  
            jpy_rate = get_jpy_rate(current_date)  
            if jpy_rate is not None:  
                print(f"Курс JPY на {current_date.strftime('%d/%m/%Y')}: {jpy_rate}")  
                write_to_google_sheets("Japan_trip", current_date, jpy_rate)  
                print("Курс успешно записан в Google Sheets.")  
            else:  
                print(f"Курс JPY на {current_date.strftime('%d/%m/%Y')} не найден.")  
            current_date += timedelta(days=1)  # Переход к следующему дню  

    except Exception as e:  
        print(f"Ошибка: {e}")

Курс JPY на 01/10/2024: 0.653502
Курс успешно записан в Google Sheets.
Курс JPY на 02/10/2024: 0.648095
Курс успешно записан в Google Sheets.
Курс JPY на 03/10/2024: 0.656835
Курс успешно записан в Google Sheets.
Курс JPY на 04/10/2024: 0.6463490000000001
Курс успешно записан в Google Sheets.
Курс JPY на 05/10/2024: 0.6467820000000001
Курс успешно записан в Google Sheets.
Курс JPY на 06/10/2024: 0.6467820000000001
Курс успешно записан в Google Sheets.
Курс JPY на 07/10/2024: 0.6467820000000001
Курс успешно записан в Google Sheets.
Курс JPY на 08/10/2024: 0.647076
Курс успешно записан в Google Sheets.
Курс JPY на 09/10/2024: 0.65008
Курс успешно записан в Google Sheets.
Курс JPY на 10/10/2024: 0.654172
Курс успешно записан в Google Sheets.
Курс JPY на 11/10/2024: 0.6513890000000001
Курс успешно записан в Google Sheets.
Курс JPY на 12/10/2024: 0.6462739999999999
Курс успешно записан в Google Sheets.
Курс JPY на 13/10/2024: 0.6462739999999999
Курс успешно записан в Google Sheets.
Курс JPY

# Код для прогрузки текущей даты 

In [15]:
# Функция для получения курса йены  
def get_jpy_rate(date):  
    formatted_date = date.strftime("%d/%m/%Y")  
    url = f"https://www.cbr.ru/scripts/XML_daily.asp?date_req={formatted_date}"  
    response = requests.get(url)  
    response.raise_for_status()  
    
    # Разбор XML  
    tree = ET.fromstring(response.content)  
    for currency in tree.findall('Valute'):  
        char_code = currency.find('CharCode').text  
        if char_code == "JPY":  
            rate = currency.find('Value').text   
            return float(rate.replace(',', '.')) / 100  # Преобразуем строку в число с плавающей запятой  
    return None  # Если курс не найден, возвращаем None  

def write_to_google_sheets(sheet_name, date, value):  
    # Авторизация Google Sheets API  
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]  
    creds = ServiceAccountCredentials.from_json_keyfile_name("/Users/kemran/Desktop/work_files/python_files/practice_files/exchange_rate/exchange-rate.json", scope)  
    client = gspread.authorize(creds)  

    # Открытие нужного Google Sheets файла  
    spreadsheet = client.open(sheet_name)  
    
    # Открытие листа по имени 'exchange_rate'  
    sheet = spreadsheet.worksheet("exchange_rate")  
    
    # Преобразование даты в формат 'YYYY-MM-DD'  
    formatted_date = date.strftime("%Y-%m-%d")  
    sheet.append_row([formatted_date, value], value_input_option="USER_ENTERED")  

# Основной блок  
if __name__ == "__main__":  
    try:  
        today = datetime.now()  # Сегодняшняя дата  
        jpy_rate = get_jpy_rate(today)  # Получение курса JPY  
        
        if jpy_rate is not None:  
            print(f"Курс JPY на {today.strftime('%d/%m/%Y')}: {jpy_rate}")  
            write_to_google_sheets("Japan_trip", today, jpy_rate)  
            print("Курс успешно записан в Google Sheets.")  
        else:  
            print(f"Курс JPY на {today.strftime('%d/%m/%Y')} не найден.")  

    except Exception as e:  
        print(f"Ошибка: {e}")


Курс JPY на 20/01/2025: 0.660095
Курс успешно записан в Google Sheets.


In [ ]:
!which python

In [11]:
pwd

'/Users/kemran/Desktop/work_files/python_files/practice_files/exchange_rate'